In [1]:
import json
from azure.eventhub import EventHubProducerClient, EventData
from dotenv import load_dotenv
import os
import socket
import random

from random import randrange

load_dotenv()
EVENT_HUB_CONN_STRING = os.getenv("EVENT_HUB_CONN_STRING") 
EVENT_HUB_NAME_IMPRESSIONS = os.getenv("EVENT_HUB_NAME_IMPRESSIONS")
EVENT_HUB_NAME_CLICKS = os.getenv("EVENT_HUB_NAME_CLICKS")

eventHubConnString = EVENT_HUB_CONN_STRING
eventHubNameImpressions = EVENT_HUB_NAME_IMPRESSIONS
eventHubNameClicks = EVENT_HUB_NAME_CLICKS

producer_impressions = EventHubProducerClient.from_connection_string(conn_str=eventHubConnString, eventhub_name=eventHubNameImpressions)
producer_clicks = EventHubProducerClient.from_connection_string(conn_str=eventHubConnString, eventhub_name=eventHubNameClicks)

hostname = socket.gethostname()

In [2]:
from utils import generateClickEvent, generateImpressionEvent

def sendToEventsHub(jsonEvent, producer):
    eventString = json.dumps(jsonEvent)
    print(eventString) 
    event_data_batch = producer.create_batch() 
    event_data_batch.add(EventData(eventString)) 
    producer.send_batch(event_data_batch)

def generateEvents(isUnsupportedBrowser = False, isAnomaly = False):
    try:
        while True:
            impressionEvent = generateImpressionEvent(isUnsupportedBrowser, isAnomaly, None)    
            sendToEventsHub(impressionEvent, producer_impressions)
            if random.randint(1, 100) > 80:
                clickEvent = generateClickEvent(impressionEvent)    
                sendToEventsHub(clickEvent, producer_clicks)
    except KeyboardInterrupt:
        producer_impressions.close()
        producer_clicks.close()


In [ ]:
#generate unsupported browser events
generateEvents(True, False)